In [ ]:
import torch
import torch.nn as nn
import numpy
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable

In [ ]:
class Unit(nn.Module):
    
    def __init__(self,in_channels,out_channels):
        super(Unit,self).__init__()
        
        self.conv = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()
        
    def forward(self,input):
        
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)
        
        return output

In [ ]:
class SimpleNet(nn.Module):
    
    def __init__(self,num_classes=10):
        
        super(SimpleNet,self).__init__()
        
        self.unit1 = Unit(in_channels=3,out_channels=32)
        self.unit2 = Unit(in_channels=32,out_channels=32)
        self.unit3 = Unit(in_channels=32,out_channels=32)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)
        
        self.net = nn.Sequential(self.unit1,self.unit2,self.unit3,self.pool1,self.unit4,
                                self.unit5,self.unit6,self.unit7,self.pool2,self.unit8,self.unit9,
                                self.unit10,self.unit11,self.pool3,self.unit12,self.unit13,self.unit14,
                                self.avgpool)
        
        self.fc = nn.Linear(in_features = 128 , out_features=num_classes)
        
        
    def forward(self,input):
        
        output = self.net(input)
        output = output.view(-1,128)
        output = self.fc(output)
        
        return output

In [ ]:
train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_set = CIFAR10(root="./data",train=True,transform=train_transformations,download=True)
train_loader = DataLoader(train_set,batch_size=32,shuffle=True,num_workers=4)

Files already downloaded and verified


In [ ]:
test_transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

])

# Load the test set, note that train is set to False
test_set = CIFAR10(root="./data", train=False, transform=test_transformations, download=True)

# Create a loder for the test set, note that both shuffle is set to false for the test loader
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)


Files already downloaded and verified


In [ ]:
cuda_avail = torch.cuda.is_available()
model = SimpleNet()

if cuda_avail:
  model.cuda()

optimizer = Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(num_epochs):
    best_acc=0
    
    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        
        for i,(images,labels) in enumerate(train_loader):
            
            if cuda_avail:
              images = Variable(images.cuda())
              labels = Variable(labels.cuda())
              
            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs,labels)
            
            loss.backward()
            
            optimizer.step()
            
            _,prediction = torch.max(outputs.data,1)
            
            train_acc += torch.sum(prediction==labels.data)
            
        train_acc = train_acc/50000
        test_acc = test()
        
        print("Epoch {} , Train Accuracy:{} , Test Accuracy:{}".format(epoch , train_acc , test_acc))

In [ ]:
def test():
    model.eval()
    test_acc = 0.0
    
    for i,(images,labels) in enumerate(test_loader):

      if cuda_avail:
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        outputs = model(images)
        _,prediction = torch.max(outputs.data,1)
        
        test_acc += torch.sum(prediction == labels.data)
        
    test_acc = test_acc/10000
    
    return test_acc
        
    

In [ ]:
train(20)

Epoch 0 , Train Accuracy:0.38830000162124634 , Test Accuracy:0.5031999945640564
Epoch 1 , Train Accuracy:0.5592199563980103 , Test Accuracy:0.6122999787330627
Epoch 2 , Train Accuracy:0.6360399723052979 , Test Accuracy:0.6541999578475952
Epoch 3 , Train Accuracy:0.6811800003051758 , Test Accuracy:0.6875
Epoch 4 , Train Accuracy:0.7226200103759766 , Test Accuracy:0.7279999852180481
Epoch 5 , Train Accuracy:0.7505199909210205 , Test Accuracy:0.774399995803833
Epoch 6 , Train Accuracy:0.7679399847984314 , Test Accuracy:0.7730000019073486
Epoch 7 , Train Accuracy:0.7849400043487549 , Test Accuracy:0.8068000078201294
Epoch 8 , Train Accuracy:0.7970799803733826 , Test Accuracy:0.7936999797821045
Epoch 9 , Train Accuracy:0.809179961681366 , Test Accuracy:0.8080999851226807
Epoch 10 , Train Accuracy:0.8208999633789062 , Test Accuracy:0.8181999921798706
Epoch 11 , Train Accuracy:0.8250399827957153 , Test Accuracy:0.820099949836731
Epoch 12 , Train Accuracy:0.8346399664878845 , Test Accuracy:0.8

In [ ]:
train(50)

Epoch 0 , Train Accuracy:0.8734999895095825 , Test Accuracy:0.859499990940094
Epoch 1 , Train Accuracy:0.8744799494743347 , Test Accuracy:0.8500999808311462
Epoch 2 , Train Accuracy:0.8753199577331543 , Test Accuracy:0.8571999669075012
Epoch 3 , Train Accuracy:0.8783599734306335 , Test Accuracy:0.866599977016449
Epoch 4 , Train Accuracy:0.8815199732780457 , Test Accuracy:0.8642999529838562
Epoch 5 , Train Accuracy:0.8816999793052673 , Test Accuracy:0.8418999910354614
Epoch 6 , Train Accuracy:0.8858199715614319 , Test Accuracy:0.8624999523162842
Epoch 7 , Train Accuracy:0.8858599662780762 , Test Accuracy:0.8673999905586243
Epoch 8 , Train Accuracy:0.8870999813079834 , Test Accuracy:0.8685999512672424
Epoch 9 , Train Accuracy:0.8910799622535706 , Test Accuracy:0.8677999973297119
Epoch 10 , Train Accuracy:0.8929199576377869 , Test Accuracy:0.858199954032898
Epoch 11 , Train Accuracy:0.89301997423172 , Test Accuracy:0.8655999898910522
Epoch 12 , Train Accuracy:0.8937999606132507 , Test Acc